1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
    - 토큰수 초과로 답변을생성하지 못할 수 있고
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래 걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

# 프롬프트를 직접 작성하는 방법 vs 허브에서 가져온 프롬프트를 사용하는 방법

In [1]:
%pip install --upgrade --quiet docx2txt langchain-community python-dotenv langchain-upstage==0.5.0


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [13]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax_with_markdown.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [3]:
document_list
len(document_list)

225

In [8]:
%pip install -r requirements.txt

  Using cached backoff-2.2.1-py3-none-any.whl.metadata (14 kB)
  Using cached chroma_hnswlib-0.7.6-cp311-cp311-macosx_11_0_arm64.whl.metadata (252 bytes)
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached humanfriendly-10.0-py2.py3-none-any.whl.metadata (9.2 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached importlib_metadata-8.4.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached langchain_chroma-0.1.4-py3-none-any.whl.metadata (1.6 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached opentelemetry_api-1.27.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached opentelemetry_exporter_otlp_proto_common-1.27.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.27.0-

In [34]:
# 임베딩
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [35]:
# 크로마에 넣기
from langchain_chroma import Chroma

#database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name='chroma-tax', persist_directory='./chroma')
database = Chroma(collection_name='chroma-tax', persist_directory="./chroma", embedding_function=embedding)

In [36]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
#retrieved_docs = database.similarity_search(query, k=3)



In [37]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [38]:
prompt = f"""
- 당신은 최고의 한국 소득세 전문가 입니다
- [Context]를 참고해서 사용자의 질문에 답변하세요

[Context]
{retrieved_docs}

Question: {query}
"""

In [39]:
ai_message = llm.invoke(prompt)

In [40]:
ai_message.content

'연봉 5천만 원인 직장인의 소득세를 계산하기 위해서는 대한민국의 소득세법에 따른 과세표준과 세율을 적용해야 합니다. 소득세 계산은 기본적으로 다음과 같은 단계로 이루어집니다:\n\n1. **총급여액 계산**: 연봉 5천만 원이 총급여액입니다.\n\n2. **근로소득공제**: 총급여액에서 근로소득공제를 차감합니다. 근로소득공제는 다음과 같은 계산식에 따라 이루어집니다.\n   - 5천만 원 이하의 경우: 3,500만 원까지는 70%, 초과분은 40% 공제\n   - 예시: 3,500만 원 × 0.7 + (5,000만 원 - 3,500만 원) × 0.4\n\n3. **과세표준 계산**: 총급여액에서 근로소득공제를 차감한 금액을 과세표준으로 조정합니다.\n   - 예시: 과세표준 = 5,000만 원 - 근로소득공제\n\n4. **기본세율 적용**: 과세표준에 따라 기본세율을 적용하여 소득세를 계산합니다.\n   - 대한민국의 소득세 기본세율은 누진세율 구조로 되어 있으며, 과세표준에 따라 세율이 달라집니다. \n   - 1,200만 원 이하: 6%\n   - 1,200만 원 초과 4,600만 원 이하: 15%\n   - 4,600만 원 초과 8,800만 원 이하: 24%\n   - 8,800만 원 초과: 35%\n\n5. **세액공제 및 기타 공제**: 계산된 소득세에서 기본세액공제, 인적공제 등 추가적인 공제를 적용합니다. \n\n6. **결정세액**: 모든 공제를 반영한 후 최종 소득세액을 결정합니다.\n\n구체적인 계산을 위해서는 개인의 소득 구조, 공제 항목, 가족 구성 등에 대한 추가 정보가 필요합니다. 따라서 위의 단계를 참고하여 개략적인 계산을 할 수 있으며, 보다 정확한 금액은 세무 전문가와 상담하거나 국세청의 세금 계산기를 활용하여 확인하는 것이 좋습니다.'

'직장인의 소득세는 여러 요인에 따라 달라질 수 있습니다. 연봉 5천만 원인 경우, 기본적으로는 근로소득세를 계산해야 합니다. 근로소득세 계산은 다음과 같은 단계로 이루어집니다.\n\n1. **근로소득 공제:**\n   - 연봉 5천만 원에 해당하는 근로소득 공제를 계산합니다. 근로소득 공제는 소득 구간에 따라 차등 적용되며, 예를 들어 연봉 5천만 원에 대해서는 약 1100만 원 정도가 공제될 수 있습니다.\n\n2. **과세표준 계산:**\n   - 총 급여에서 근로소득 공제를 뺀 금액이 과세표준이 됩니다. 예를 들어, 5천만 원 - 1100만 원 = 3900만 원이 과세표준이 될 수 있습니다.\n\n3. **기본세율 적용:**\n   - 과세표준에 따라 기본세율을 적용합니다. 한국의 소득세율은 과세표준에 따라 6%, 15%, 24%, 35%, 38%, 40%, 42%, 45%로 구분됩니다.\n   - 예를 들어, 과세표준 3900만 원에 대해 1200만 원까지는 6%, 초과분은 15%가 적용됩니다.\n\n4. **세액 공제 및 감면:**\n   - 산출된 세액에서 다양한 세액 공제 및 감면 사항을 적용합니다. 예를 들어, 인적 공제, 보험료 공제, 의료비 공제 등이 있을 수 있습니다.\n\n이 모든 과정을 거치면 최종적으로 납부해야 할 소득세가 계산됩니다. 정확한 금액을 알기 위해서는 세법에 따른 구체적인 계산이 필요합니다. 세금 계산기나 전문가의 도움을 받는 것이 좋습니다.'

In [41]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

/Users/hs/.pyenv/versions/inflearn-llm-application/lib/python3.11/site-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [42]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt":prompt}
)

In [43]:
ai_message = qa_chain({"query":query})

In [44]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 직장인의 소득세는 기본세율이 적용됩니다. 그러나 구체적인 세율은 제공된 정보에서 확인할 수 없습니다. 일반적으로 한국의 소득세율은 소득 구간에 따라 차등 적용되므로, 정확한 금액을 계산하려면 소득세율표를 참고해야 합니다.'}